# Fixed-Length Few-Shot


In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    {
        "question": "What is the capital of France?",
        "answer": "Paris"
    },
    {
        "question": "Who wrote the play 'Romeo and Juliet'?",
        "answer": "William Shakespeare"
    },
    {
        "question": "What is the chemical symbol for water?",
        "answer": "H2O"
    },
    {
        "question": "What is the largest planet in our solar system?",
        "answer": "Jupiter"
    },
    {
        "question": "What year did the Titanic sink?",
        "answer": "1912"
    },
    {
        "question": "What is the fastest land animal?",
        "answer": "Cheetah"
    },
    {
        "question": "Who was the first President of the United States?",
        "answer": "George Washington"
    },
    {
        "question": "What is the square root of 64?",
        "answer": "8"
    },
    {
        "question": "Which element has the atomic number 1?",
        "answer": "Hydrogen"
    },
    {
        "question": "What is the largest ocean on Earth?",
        "answer": "Pacific Ocean"
    },
    {
        "question": "What is the smallest country in the world?",
        "answer": "Vatican City"
    },
    {
        "question": "Who painted the Mona Lisa?",
        "answer": "Leonardo da Vinci"
    }
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}")
    ]
)

In [2]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

print(few_shot_prompt.format())

Human: What is the capital of France?
AI: Paris
Human: Who wrote the play 'Romeo and Juliet'?
AI: William Shakespeare
Human: What is the chemical symbol for water?
AI: H2O
Human: What is the largest planet in our solar system?
AI: Jupiter
Human: What year did the Titanic sink?
AI: 1912
Human: What is the fastest land animal?
AI: Cheetah
Human: Who was the first President of the United States?
AI: George Washington
Human: What is the square root of 64?
AI: 8
Human: Which element has the atomic number 1?
AI: Hydrogen
Human: What is the largest ocean on Earth?
AI: Pacific Ocean
Human: What is the smallest country in the world?
AI: Vatican City
Human: Who painted the Mona Lisa?
AI: Leonardo da Vinci


In [3]:
from langchain_core.output_parsers import StrOutputParser
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You only need to return a single answer without any explanation"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

model = ChatOpenAI()

chain = final_prompt | model | StrOutputParser()

response = chain.invoke({
    "input": "What is the capital of Malaysia?"
})

print(response)

Kuala Lumpur


# Selecting Few-Shot Examples by Length


In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
import tiktoken

In [9]:
examples = [
    {
        "input": "James",
        "output": "James was a brilliant inventor who spent years designing a machine that could communicate with animals. One day, his machine finally worked, and he had a conversation with a wise old owl who shared secrets of the forest with him."
    },
    {
        "input": "Curry",
        "output": "Curry was a talented chef known for his spicy dishes that people from all over the world traveled to taste. One evening, a mysterious customer challenged Curry to create the hottest dish imaginable, which led to a recipe that became legendary."
    },
    {
        "input": "Alice",
        "output": "Alice discovered a hidden garden behind her grandmother's old house. Inside the garden, she found flowers that glowed in the dark and led her to a portal that transported her to a magical realm where she met talking animals and enchanted creatures."
    },
    {
        "input": "Bob",
        "output": "Bob, an ordinary office worker, found a dusty old map in a forgotten drawer at work. Following the map, he embarked on an adventure that took him through dangerous jungles and across vast deserts, eventually leading him to a treasure that changed his life."
    },
    {
        "input": "Sarah",
        "output": "Sarah was a scientist who dedicated her life to studying the stars. One night, she detected a signal from deep space that no one had ever heard before. Following the signal, she made contact with an alien civilization and became Earth's first ambassador to another world."
    },
    {
        "input": "Liam",
        "output": "Liam was a skilled swordsman in a kingdom plagued by dragons. When a massive dragon threatened his village, Liam bravely faced the beast, defeating it with a clever trap he had devised using only his wits and his sword."
    },
    {
        "input": "Emma",
        "output": "Emma was an explorer who had always been fascinated by the mysteries of the ocean. On one of her dives, she discovered an ancient underwater city and unlocked the secrets of a lost civilization that had once ruled the seas."
    },
    {
        "input": "Noah",
        "output": "Noah was a young boy with a passion for building robots. One day, he created a robot with artificial intelligence that became his best friend. Together, they went on adventures, solving mysteries and helping people in their town."
    },
    {
        "input": "Sophia",
        "output": "Sophia was a skilled archer living in a peaceful village. When her home was threatened by invaders, she rallied the villagers and led a defense, using her expert marksmanship to protect her people and drive away the enemy."
    },
    {
        "input": "Mason",
        "output": "Mason was an architect who dreamed of building a city in the sky. After years of hard work and collaboration with engineers and scientists, he created floating islands that housed a beautiful, futuristic city that attracted people from all over the world."
    }
]


example_prompt = PromptTemplate(
    template="""Character: {input} \n Story: {output}""",
    input_variables=["input", "output"]
)

In [16]:
def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens


example_selector = LengthBasedExampleSelector(
    example_prompt=example_prompt,
    examples=examples,
    max_length=1000,
    get_text_length=num_tokens_from_string
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="""Generate the story for {character}, based on the list of Character/Story pair provided in the context""",
    suffix="Character: {character} \n Story:",
    input_variables=["character"]
)

formatted_prompt = dynamic_prompt.format(character="Jordan")

model = ChatOpenAI()

response = model.invoke([SystemMessage(content=formatted_prompt)])
print(response.content)

Jordan was an aspiring musician with a deep connection to nature. While hiking one day, he stumbled upon a hidden cave deep in the forest. Inside the cave, he found an ancient flute made of mystical wood. When he played the flute, the forest came alive with the sound of animals singing and dancing to his music. Jordan soon realized that he had the power to communicate with the animals through his flute, and together they created beautiful melodies that echoed through the woods, bringing harmony to the creatures of the forest. Word of Jordan's unique gift spread far and wide, and people from all over came to listen to his enchanting music, feeling a sense of peace and unity with nature whenever they heard him play.


# Saving and Loading LLM Prompts


In [17]:
from langchain_core.prompts import PromptTemplate, load_prompt

prompt = PromptTemplate(
    template="""Translate this sentence from English to Chinese.
        Sentence: {sentence}
    """,
    input_variables=["sentence"]
)

prompt.save("translation_prompt.json")

prompt = load_prompt("translation_prompt.json")
print(prompt)

input_variables=['sentence'] template='Translate this sentence from English to Chinese.\n        Sentence: {sentence}\n    '
